In [11]:
raw_text = ""
with open("verdict.txt", "r") as f:
    raw_text = f.read()

import re
class SimpleTokenizer:
  def __init__(self, raw_text):
    prepocesssed = re.split(r"([,.:;?_()!<>\"]|--|\s)", raw_text)
    preprocessed = [item.strip() for item in prepocesssed if item.strip()]
    self.vocab = sorted(set(preprocessed))
    self.vocab.append("<|endoftext|>")
    self.str_to_int = {token:integer for integer, token in enumerate(self.vocab)}
    self.int_to_str = {i:s for s, i in self.str_to_int.items()}
    self.int_to_str[-1] = "<|unk|>"
  def encode(self, raw):
    preprocessed = re.split(r"([,.:;?_()!\"]|--|\s)", raw)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    res = []
    for i in preprocessed:
      try:
        res.append(self.str_to_int[i])
      except KeyError:  
        res.append(-1)
    return res
  def decode(self, ids):
     return " ".join([self.int_to_str[i] for i in ids])
  
     

In [12]:
tokenizer = SimpleTokenizer(raw_text)
a1 = tokenizer.encode("Hello")

t1 = "Hello, How are you"
t2 = "she had been Dragged"

t3 = " <|endoftext|> ".join((t1, t2))
a1 = tokenizer.encode(t3)
print(tokenizer.decode(a1))
print(a1)

<|unk|> , How are you <|endoftext|> she had been <|unk|>
[-1, 12, 61, 187, 1148, 1154, 896, 531, 226, -1]


In [10]:
len(tokenizer.vocab)

1155

In [1]:
import importlib
import tiktoken

print(importlib.metadata.version("tiktoken"))

0.12.0


In [8]:
tokenizer = tiktoken.get_encoding("gpt2")
text = (
    "Akwirw ier"
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
integers

[33901, 86, 343, 86, 220, 959]

In [9]:
tokenizer.decode(integers)

'Akwirw ier'

In [ ]:
import torch
from torch.utils.data import Dataset


class GPDdatasetV1(Dataset):


    def __init__(self, text_file, tokenizer, block_size, stride=1):
        self.block_size = block_size
        self.stride = stride
        self.tokenizer = tokenizer
        with open(text_file, "r", encoding="utf-8") as f:
            text = f.read()
        self.tokens = tokenizer.encode(text)

        self.num_samples = (
            len(self.tokens) - block_size - 1
        ) // stride + 1

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        start = idx * self.stride
        end = start + self.block_size

        input_ids = self.tokens[start:end]
        target_ids = self.tokens[start + 1:end + 1]

        return (
            torch.tensor(input_ids, dtype=torch.long),
            torch.tensor(target_ids, dtype=torch.long),
        )

In [ ]:
class GPTDatasetv1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(txt, allowed_special  = {"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            target_chunk = token_ids[i +1, i+max_length + 1]
            self.input_ids.append(torch.tensor(self.input_ids))
            self.output_ids.append(torch.tensor(self.output_ids))
        def __len__(self):
            return len(self.input_ids)
        
        def __getitem__(self, idx):
            return self.input_ids[idx], self.target_ids[idx]


import torch
from torch.utils.data import DataLoader


def get_dataloader(
    text_file,
    tokenizer,
    block_size,
    batch_size,
    stride=1,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    drop_last=True,
):
    dataset = GPDdatasetV1(
        text_file=text_file,
        tokenizer=tokenizer,
        block_size=block_size,
        stride=stride,
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
    )